In [ ]:

import pandas
import random
import numpy as np
import copy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

class ACO:
    def __init__(self, data, maxIteration, antNumber, cc, Q, e):
        self.data = data
        self.fp = [cc] * (len(data.columns) - 1)
        self.maxIteration = maxIteration
        self.ants = []
        self.size = len(data.columns) - 1
        self.antNumber = antNumber
        self.Q = Q
        self.bestScore = 0
        self.result = []
        self.evaporate = e
        self.colonyMax = 0
        self.colonyQuality = 0

    def constructSolution(self, ant):
        featureSetIndex = []
        for j in range(self.size):
            decision = random.random()
            if decision < self.fp[j] / 2.0:
                featureSetIndex.append(1)
            else:
                featureSetIndex.append(0)
#         print(featureSetIndex)
#         featureSetIndex.append(1)
        features = []
        for i, obj in enumerate(featureSetIndex):
            if obj:
                features.append(i)

        features.append(12)
#         print(features)
        newdata = self.data.iloc[:, features]
#         print(newdata.columns)
        if sum(featureSetIndex) == 0:
            score = 0.5
        else:
#             X_train, X_test, y_train, y_test = train_test_split(newdata, self.data['num'], test_size=0.2, random_state=42)
            X = newdata.drop(columns=['num'])
            y= newdata['num']
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            clf = ExtraTreesClassifier(random_state=42)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            score = accuracy_score(y_test, y_pred)
#             print(newdata.columns,score)
        ant.val = score
        ant.subsets = copy.deepcopy(featureSetIndex)
        ant.dataset = newdata
        return ant


    def ApplyLocalSearch(self):
        maxScore = 0
        maxSet = []
        maxdataset = []
        for a in self.ants:
            if maxScore < a.val or (maxScore == a.val and (maxSet and sum(a.subsets) < sum(maxSet))):
                maxScore = a.val
                maxSet = a.subsets
                maxdataset = a.dataset

        if self.bestScore <= maxScore or (maxScore == self.bestScore and (self.result and sum(maxSet) < sum(self.result))):
            self.bestScore = maxScore
            self.result = maxSet

        #print(maxScore)
        self.colonyMax += maxScore
#         print('i',maxSet,maxScore)
        return maxSet, maxScore, maxdataset

    def UpdatePheromones(self,bestSet, bestScore):
        for i,v in enumerate(bestSet):
            self.fp[i] = self.fp[i]*self.evaporate
            if v == 1:
                weight = (bestScore-0.5)*2
                self.fp[i] = self.fp[i] + self.Q*weight

    def simulate(self):
        for s in range(self.maxIteration):
            for i in range(self.antNumber):
                ant = Ant()
                ant = self.constructSolution(ant)
                self.ants.append(ant)
            bestSet, bestScore, dataset = self.ApplyLocalSearch()
            self.UpdatePheromones(bestSet, bestScore)
            self.ants = []

        shortFeaturesName = list(self.data.columns.values)
        bestFeatureName = []
        for ind, obj in enumerate(self.result):
            if obj:
                bestFeatureName.append(shortFeaturesName[ind + 1])
        new_file_path = r'C:\Users\THASNEEM FATHIMA\Downloads\aco2.csv'
        dataset.to_csv(new_file_path, index=False)
        return ["Best", bestSet, dataset, self.bestScore]

    def run(self):
        for s in range(self.maxIteration):
            for i in range(self.antNumber):
                ant = Ant()
                ant = self.constructSolution(ant)
                self.ants.append(ant)
            bestSet, bestScore = self.ApplyLocalSearch()
            self.UpdatePheromones(bestSet, bestScore)
            self.ants = []

        shortFeaturesName = list(self.data.columns.values)
        bestFeatureName = []
        for ind, obj in enumerate(self.result):
            if obj:
                bestFeatureName.append(shortFeaturesName[ind + 1])

        self.colonyQuality = self.colonyMax / self.maxIteration # the overall ant colony quality over iterations
        return self.bestScore, self.result, self.colonyQuality

class Ant:
    def __init__(self):
        self.subsets = []
        self.val = 0
        self.dataset = []

data = pd.read_csv(r'C:\Users\THASNEEM FATHIMA\Downloads\fcbfdata.csv')
acoModel = ACO(data,maxIteration=30,antNumber=30,cc=1,Q=0.1,e=0.95)
result = acoModel.simulate()
print('Result:', result)

Result: ['Best', [1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],           age        cp  trestbps      chol  restecg    thalch   oldpeak  \
0    0.489408  0.299439  0.316918  0.326599      0.0  0.749192  0.276246   
1    0.545558  0.000000  0.380050  0.366284      0.0  0.299136  0.252751   
2    0.615378  0.606874  0.197733  0.276693      0.0  0.694964  0.395993   
3    0.135869  0.000000  0.252442  0.215091      0.5  0.442779  0.256535   
4    0.505407  0.000000  0.589802  0.372818      0.0  0.519713  0.230092   
..        ...       ...       ...       ...      ...       ...       ...   
683  0.644220  0.000000  0.346804  0.245977      0.5  0.673315  0.005793   
684  0.297821  0.016412  0.519012  0.318826      0.5  0.763359  0.234788   
685  0.562500  0.000000  0.245283  0.000000      0.5  0.374046  0.241935   
686  0.637624  0.309789  0.301115  0.206336      0.0  0.554102  0.498201   
687  0.676074  0.763435  0.428485  0.243879      0.0  0.683327  0.408953   

        slope        ca      tha